In [2]:
import torch
import gradio as gr
import torchvision.transforms as transforms
from PIL import Image
from model import UNet
import numpy as np


ImportError: cannot import name 'Undefined' from 'pydantic.fields' (/home/jedld/.local/lib/python3.10/site-packages/pydantic/fields.py)

In [ ]:

INPUT_RESOLUTION = (256,256)

# Load the trained model
model_path = "best_model.pth"
model = UNet(in_channels=3, out_channels=3)  # Assuming the model is for RGB images
model.load_state_dict(torch.load(model_path))
model.eval()

def inpaint_image(original_image: Image.Image, sketch: np.ndarray) -> Image.Image:
    # Combine original image with the sketch
    sketch_image = Image.fromarray((sketch * 255).astype(np.uint8))
    combined_image = Image.blend(original_image, sketch_image, alpha=0.5)

    # Convert image to tensor and normalize it
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Assuming normalization values
    ])
    input_tensor = transform(combined_image).unsqueeze(0)  # Add batch dimension
    
    # Feed the tensor to the model
    with torch.no_grad():
        output_tensor = model(input_tensor)
    
    # Convert the tensor back to image
    output_image = transforms.ToPILImage()(output_tensor.squeeze(0))
    return output_image

# Set up Gradio interface
interface = gr.Interface(
    fn=inpaint_image,
    inputs=gr.inputs.Image(type="file"),
    outputs="image",
    live=True,
    title="UNet Inpainting",
    description="Upload an image with black pixels to inpaint using UNet."
)

interface.launch()
